In [1]:
import pandas as pd

# Obtaining the postal codes

In [2]:
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

I will read all the tables on that web page and check, which ones to keep

In [42]:
df = pd.read_html(wiki_url)

In [6]:
len(df)

3

In [7]:
df[0].head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [8]:
df[1].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,Canadian postal codes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NL NS PE NB QC ON MB SK AB BC NU/NT YT A B C E...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
3,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


In [9]:
df[2].head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NL,NS,PE,NB,QC,QC,QC,ON,ON,ON,ON,ON,MB,SK,AB,BC,NU/NT,YT
1,A,B,C,E,G,H,J,K,L,M,N,P,R,S,T,V,X,Y


From here, the first dataframe caontains the required information, so I will drop the rest

In [43]:
df = df[0]

Drop the records where Borough is not assigned

In [44]:
df = df.loc[df['Borough'] != 'Not assigned',:]

Fill the missing Neighborhoods

In [45]:
df.loc[pd.isnull(df['Neighborhood']),:]

,Postal code,Borough,Neighborhood


Looks like there are no such cases, so no conversion is required

Now we need to group matching postal codes

In [37]:
def merger(names):
    unique_names = []
    for name in names:
        if name in unique_names:
            continue
        else:
            unique_names.append(name)
    return ', '.join(unique_names)

In [46]:
df = df.groupby(['Postal code'], as_index=False).agg(merger)

In [47]:
df.shape

(103, 3)

# Adding coordinates

In [49]:
! pip install geocoder

     |████████████████████████████████| 102kB 17.0MB/s ta 0:00:01


In [50]:
import geocoder # import geocoder

In [55]:
def get_coordinates(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [56]:
df_test = df.iloc[0:2,:]

In [57]:
df_test.shape

(2, 3)

In [58]:
df_test.loc[:,['Latitude', 'Longitude']] = df['Postal code'].apply(get_coordinates)

KeyboardInterrupt: 

Didn't work out in 10 minutes, so I will use the provided url

In [59]:
coords_url = 'https://cocl.us/Geospatial_data'

In [61]:
coords_dfs = pd.read_csv(coords_url)

In [63]:
coords_dfs.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Fix column names and perform merge

In [67]:
df.columns = ['Postal Code', 'Borough', 'Neighborhood']

In [70]:
df = df.merge(coords_dfs, how='left', on='Postal Code')

In [71]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
